# Affiliation and Partner Affiliations Analysis with MAG

## Prerequisites

Complete these tasks before you begin this tutorial:

- Setting up provisioning of Microsoft Academic Graph to an Azure blob storage account. See [Get Microsoft Academic Graph on Azure storage](https://docs.microsoft.com/academic-services/graph/get-started-setup-provisioning).
- Setting up Azure Databricks service. See [Set up Azure Databricks](https://docs.microsoft.com/academic-services/graph/get-started-setup-databricks).

## Gather the information

Before you begin, you should have these items of information:

- The name of your Azure Storage (AS) account containing MAG dataset from [Get Microsoft Academic Graph on Azure storage](https://docs.microsoft.com/academic-services/graph/get-started-setup-provisioning#note-azure-storage-account-name-and-primary-key).
- The access key of your Azure Storage (AS) account from [Get Microsoft Academic Graph on Azure storage](https://docs.microsoft.com/academic-services/graph/get-started-setup-provisioning#note-azure-storage-account-name-and-primary-key).
- The name of the container in your Azure Storage (AS) account containing MAG dataset.

## Import notebooks

- [Import](https://docs.databricks.com/user-guide/notebooks/notebook-manage.html#import-a-notebook) samples/pyspark/MagClass.py in MAG dataset under your working folder.
- [Import](https://docs.databricks.com/user-guide/notebooks/notebook-manage.html#import-a-notebook) this notebook under the same folder.

### Initialize storage account and container details

  | Variable  | Value | Description  |
  | --------- | --------- | --------- |
  | AzureStorageAccount | Replace **`<AzureStorageAccount>`** | This is the Azure Storage account containing MAG dataset. |
  | AzureStorageAccessKey | Replace **`<AzureStorageAccessKey>`** | This is the Access Key of the Azure Storage account. |
  | MagContainer | Replace **`<MagContainer>`** | This is the container name in Azure Storage account containing MAG dataset, usually in the form of `mag-yyyy-mm-dd`. |
  | OutputContainer | Replace **`<OutputContainer>`** | This is the container name in Azure Storage account where the output goes to, this container needs to be created before running this script. |

In [0]:
AzureStorageAccount = '<AzureStorageAccount>'
AzureStorageAccessKey = '<AzureStorageAccessKey>'
MagContainer = '<MagContainer>'
OutputContainer = '<OutputContainer>'

### Define MicrosoftAcademicGraph class

Run the MagClass notebook to define MicrosoftAcademicGraph class.

In [0]:
%run "./MagClass"

### Create a MicrosoftAcademicGraph instance to access MAG dataset
Use account=AzureStorageAccount, key=AzureStorageAccessKey, container=MagContainer.

In [0]:
mag = MicrosoftAcademicGraph(account=AzureStorageAccount, key=AzureStorageAccessKey, container=MagContainer)

### Create a AzureStorageUtil to access other Azure Storage files
Use account=AzureStorageAccount, key=AzureStorageAccessKey, container=OutputContainer.

In [0]:
asu = AzureStorageUtil(account=AzureStorageAccount, key=AzureStorageAccessKey, container=OutputContainer)

In [0]:
from pyspark.sql.functions import concat, lit, log, when, count, sum

### Step 1. Get target affiliation

#### Load Affiliations data

In [0]:
affiliations = mag.getDataframe('Affiliations')

# Peek the result
display(affiliations.head(5))

AffiliationId,Rank,NormalizedName,DisplayName,GridId,OfficialPage,WikiPage,PaperCount,PaperFamilyCount,CitationCount,Iso3166Code,Latitude,Longitude,CreatedDate
20455151,9877,air liquide,Air Liquide,grid.476009.c,https://web.archive.org/web/20100205175402/http://airliquide.com/en/home.html,http://en.wikipedia.org/wiki/Air_Liquide,7692,5632,60056,GB,52.50359344482422,-1.8051600456237793,2016-06-24
24386293,13932,hellenic national meteorological service,Hellenic National Meteorological Service,null,http://www.hnms.gr/hnms/english/index_html,http://en.wikipedia.org/wiki/Hellenic_National_Meteorological_Service,86,86,1975,GR,37.976139068603516,23.736400604248047,2016-06-24
32956416,12969,catholic university of the west,Catholic University of the West,grid.448708.7,http://www.uco.fr/,http://en.wikipedia.org/wiki/Catholic_University_of_the_West,363,354,4316,FR,47.4647216796875,-0.5486099720001221,2016-06-24
35926432,11668,mackay medical college,Mackay Medical College,grid.452449.a,http://www.mmc.edu.tw/,http://en.wikipedia.org/wiki/Mackay_Medical_College,1510,1504,14671,TW,25.25436019897461,121.49508666992188,2016-06-24
37448385,11875,chinese people s public security university,Chinese People's Public Security University,null,http://www.ppsuc.edu.cn/,http://en.wikipedia.org/wiki/People's_Public_Security_University_of_China,1792,1786,2613,CN,39.90468978881836,116.40717315673828,2016-06-24


#### Extract AffiliationId and DisplayName for the affiliation

In [0]:
# Extract the AffiliationId for Microsoft
inputAffiliationName = 'microsoft'
orgAffiliation = affiliations.where(affiliations.NormalizedName == inputAffiliationName).select(affiliations.AffiliationId, affiliations.DisplayName)

# Peek the result
display(orgAffiliation)

# Save result
asu.save(orgAffiliation, 'Affiliation/Affiliation.tsv', coalesce=True)

AffiliationId,DisplayName
1290206253,Microsoft


### Step 2. Get Venues

#### Load ConferenceSeries data

In [0]:
conferences = mag.getDataframe('ConferenceSeries')

# Peek result
display(conferences.head(5))

ConferenceSeriesId,Rank,NormalizedName,DisplayName,PaperCount,PaperFamilyCount,CitationCount,CreatedDate
1134804816,12797,ICIDS,International Conference on Interactive Digital Storytelling,608,607,2696,2016-06-24
1165160117,14799,SWAT4LS,Semantic Web Applications and Tools for Life Sciences,81,81,197,2016-06-24
1192093291,12251,TRIDENTCOM,Testbeds and Research Infrastructures for the DEvelopment of NeTworks and COMmunities,570,570,5047,2016-06-24
1199066382,10256,BIOINFORMATICS,International Conference on Bioinformatics,9226,9226,14451,2016-06-24
1201746639,15536,AIS,Autonomous and Intelligent Systems,165,165,963,2016-06-24


#### Load Journals data

In [0]:
journals = mag.getDataframe('Journals').drop('Publisher')

# Peek result
display(journals.head(5))

JournalId,Rank,NormalizedName,DisplayName,Issn,Webpage,PaperCount,PaperFamilyCount,CitationCount,CreatedDate
465895,12370,eureka,Eureka,1342-5641,http://www.archim.org.uk/eureka/,1118,1118,187,2016-06-24
1137746,12716,the artist and journal of home culture,The Artist and Journal of Home Culture,2151-4879,null,562,562,480,2016-06-24
2978343,15329,cumberland law review,Cumberland Law Review,0360-8298,http://www.cumberlandlawreview.com/,47,47,45,2016-06-24
3010151,14770,comparative medicine east and west,Comparative Medicine East and West,0147-2917,null,23,23,211,2016-06-24
3164724,9968,physiological measurement,Physiological Measurement,0967-3334,null,3456,3456,78322,2016-06-24


#### Union ConferenceSeries and Journals as Venues

In [0]:
conferences = conferences \
    .select(conferences.ConferenceSeriesId, conferences.DisplayName, conferences.NormalizedName) \
    .selectExpr('ConferenceSeriesId as VId', 'DisplayName as VenueName', 'NormalizedName as VenueShortName')

journals = journals \
    .select(journals.JournalId, journals.DisplayName, journals.NormalizedName) \
    .selectExpr('JournalId as VId', 'DisplayName as VenueName', 'NormalizedName as VenueShortName')

venues = conferences.union(journals)

# Count number of rows in result
print('Number of rows in venues: {}'.format(venues.count()))

# Peek result
display(venues.head(5))

# Save result
asu.save(venues, 'Affiliation/Venues.tsv', coalesce=True)

Number of rows in venues: 53525

VId,VenueName,VenueShortName
1134804816,International Conference on Interactive Digital Storytelling,ICIDS
1165160117,Semantic Web Applications and Tools for Life Sciences,SWAT4LS
1192093291,Testbeds and Research Infrastructures for the DEvelopment of NeTworks and COMmunities,TRIDENTCOM
1199066382,International Conference on Bioinformatics,BIOINFORMATICS
1201746639,Autonomous and Intelligent Systems,AIS


### Step 3. Get PaperAuthorAffiliation for the affiliation

#### Load PaperAuthorAffiliations data

In [0]:
paperAuthorAffiliations = mag.getDataframe('PaperAuthorAffiliations')

# Peek result
display(paperAuthorAffiliations.head(5))

PaperId,AuthorId,AffiliationId,AuthorSequenceNumber,OriginalAuthor,OriginalAffiliation
9,2632942543,null,1,Victoriano Perruca Albadalejo,null
15,199142497,null,1,Robert Münscher,"Heidelberg, Deutschland"
15,680395887,null,2,Julia Hormuth,"Reutlingen, Deutschland"
23,1243978490,79576946,1,Eric T Stoopler,"Department of Oral Medicine, University of Pennsylvania School of Dental Medicine, Philadelphia, PA 19104, USA."
23,2582258949,null,3,Arthur S Kuperstein,null


#### Get PaperAuthorAffiliations for the affiliation

In [0]:
orgPaperAuthorAffiliation = paperAuthorAffiliations \
    .join(orgAffiliation, paperAuthorAffiliations.AffiliationId == orgAffiliation.AffiliationId, 'inner') \
    .select(paperAuthorAffiliations.PaperId, paperAuthorAffiliations.AuthorId, \
            orgAffiliation.AffiliationId, paperAuthorAffiliations.AuthorSequenceNumber)

# Count number of rows in result
print('Number of rows in PaperAuthorAffiliation: {}'.format(orgPaperAuthorAffiliation.count()))

# Peek result
display(orgPaperAuthorAffiliation.head(5))

# Save result
asu.save(orgPaperAuthorAffiliation, 'Affiliation/PaperAuthorAffiliationRelationships.tsv')

Number of rows in PaperAuthorAffiliation: 221410

PaperId,AuthorId,AffiliationId,AuthorSequenceNumber
15883,135218249,1290206253,1
15883,2120803867,1290206253,2
15883,2128381254,1290206253,3
15883,2150137935,1290206253,4
73910,2279432523,1290206253,1


### Step 4. Get author and paper details

#### Get authors for the affiliation

In [0]:
#Load data from previous output
orgPaperAuthorAffiliation = asu.load('Affiliation/PaperAuthorAffiliationRelationships.tsv')

orgAuthorIds = orgPaperAuthorAffiliation.select(orgPaperAuthorAffiliation.AuthorId).distinct()

#Load Authors data
authors = mag.getDataframe('Authors')

# Get all author details
orgAuthors = authors \
    .join(orgAuthorIds, authors.AuthorId == orgAuthorIds.AuthorId, 'inner') \
    .select(orgAuthorIds.AuthorId, authors.DisplayName.alias('AuthorName'))

# Peek result
display(orgAuthors.head(5))

# Save result
asu.save(orgAuthors, 'Affiliation/Authors.tsv')

AuthorId,AuthorName
53423,Yannai A. Gonczarowski
720112,Sergei Gringauze
1665409,Rogier Dittner
2364515,Steven J. Altschuler
4011424,Nicolae Surpatanu


#### Get papers for the affiliation

In [0]:
#Load Papers data
papers = mag.getDataframe('Papers')

papers = papers.withColumn('Prefix', lit('https://academic.microsoft.com/#/detail/'))

# Get all paper details
orgPaperIds = orgPaperAuthorAffiliation.select(orgPaperAuthorAffiliation.PaperId).distinct()

orgPapers = papers \
    .join(orgPaperIds, papers.PaperId == orgPaperIds.PaperId) \
    .where(papers.Year >= 1991) \
    .select(papers.PaperId, papers.PaperTitle.alias('Title'), papers.EstimatedCitation.alias('CitationCount'), \
            papers.Date, when(papers.DocType.isNull(), 'Not available').otherwise(papers.DocType).alias('PublicationType'), \
            log(papers.Rank).alias('LogProb'), concat(papers.Prefix, papers.PaperId).alias('Url'), \
            when(papers.ConferenceSeriesId.isNull(), papers.JournalId).otherwise(papers.ConferenceSeriesId).alias('VId'), \
            papers.Year)

# Optional: Count number of rows in result
print('Number of rows in orgPapers: {}'.format(orgPapers.count()))

# Peek result
display(orgPapers.head(5))

# Save result
asu.save(orgPapers, 'Affiliation/Papers.tsv')

Number of rows in orgPapers: 85652

PaperId,Title,CitationCount,Date,PublicationType,LogProb,Url,VId,Year
2568259326,the unsplittable stable marriage problem,0,2006-01-01,Not available,9.972360416822504,https://academic.microsoft.com/#/detail/2568259326,null,2006
1535764483,the unsplittable stable marriage problem,18,2006-08-01,Conference,9.935664284231349,https://academic.microsoft.com/#/detail/1535764483,2755269626,2006
2914429498,proceedings of the the 1st acm workshop on continuous archival and retrieval of personal experiences,0,2004-10-15,Conference,10.036005932236272,https://academic.microsoft.com/#/detail/2914429498,2758214222,2004
1536284211,proceedings of the the 1st acm workshop on continuous archival and retrieval of personal experiences,6,2004-10-15,Conference,9.995793223320154,https://academic.microsoft.com/#/detail/1536284211,1135237122,2004
1580275452,semantically annotated provenance in the life science grid,4,2009-10-25,Conference,9.951753769945617,https://academic.microsoft.com/#/detail/1580275452,1155608529,2009


### Step 5. Get fields of study for the affiliation

#### Get PaperFieldsOfStudy

In [0]:
#Load data from previous output
orgPapers = asu.load('Affiliation/Papers.tsv')

# Load FieldsOfStudy data
fieldsOfStudy = mag.getDataframe('FieldsOfStudy')

# Load PaperFieldsOfStudy data
paperFieldsOfStudy = mag.getDataframe('PaperFieldsOfStudy')

# Get Paper-Field-of-Study relationships for the input organization
orgPaperFieldsOfStudy = paperFieldsOfStudy \
    .join(orgPapers, paperFieldsOfStudy.PaperId == orgPapers.PaperId, 'inner') \
    .select(orgPapers.PaperId, paperFieldsOfStudy.FieldOfStudyId)

# Optional: peek result
display(orgPaperFieldsOfStudy.head(5))

# Save result
asu.save(orgPaperFieldsOfStudy, 'Affiliation/PaperFieldOfStudyRelationships.tsv')

PaperId,FieldOfStudyId
199255522,185954173
199255522,83479923
199255522,116081451
199255522,570499
199255522,129353971


#### Get FieldsOfStudy

In [0]:
# Get all field-of-study Ids for the input organization
orgFieldsOfStudyIds = orgPaperFieldsOfStudy.select(orgPaperFieldsOfStudy.FieldOfStudyId).distinct()

# Get all field-of-study details for the input organization
orgFieldsOfStudy = fieldsOfStudy \
    .join(orgFieldsOfStudyIds, fieldsOfStudy.FieldOfStudyId == orgFieldsOfStudyIds.FieldOfStudyId, 'inner') \
    .select(orgFieldsOfStudyIds.FieldOfStudyId, fieldsOfStudy.Level.alias('FieldLevel'), fieldsOfStudy.DisplayName.alias('FieldName'))

# Optional: peek result
display(orgFieldsOfStudy.head(5))

# Save result
asu.save(orgFieldsOfStudy, 'Affiliation/FieldsOfStudy.tsv')

FieldOfStudyId,FieldLevel,FieldName
2779313563,2,On Language
2992447835,2,Risk groups
207912722,2,Signal generator
2780719635,2,Flavor
3017490568,2,Automated algorithm


### Step 6. Get partner data

#### Load data previousely generated

In [0]:
# Get all paper details for the input organization from previous output
orgPapers = asu.load('Affiliation/Papers.tsv')

# Get all Paper-Author-Affiliation relationships for the input organization from previous output
orgPaperAuthorAffiliation = asu.load('Affiliation/PaperAuthorAffiliationRelationships.tsv')

# Get all paper-author-affiliation relationships
paperAuthorAffiliations = mag.getDataframe('PaperAuthorAffiliations')

# Get all affiliation details
affiliations = mag.getDataframe('Affiliations')

# Get all author details
authors = mag.getDataframe('Authors')

#### Get partner paper-author-affiliation relationships

In [0]:
# Get all Paper-Author-Affiliation relationships for papers published by the input organization
orgAllPaperAuthorAffiliations = paperAuthorAffiliations \
    .join(orgPapers, paperAuthorAffiliations.PaperId == orgPapers.PaperId, 'inner') \
    .select(orgPapers.PaperId, paperAuthorAffiliations.AuthorId, \
            paperAuthorAffiliations.AffiliationId, paperAuthorAffiliations.AuthorSequenceNumber)

# Get partner Paper-Author-Affiliation relationships by excluding those relationships of the input organization
partnerPaperAuthorAffiliation = orgAllPaperAuthorAffiliations.subtract(orgPaperAuthorAffiliation)
display(partnerPaperAuthorAffiliation.head(5))

# Save result
asu.save(partnerPaperAuthorAffiliation, 'Affiliation/PartnerPaperAuthorAffiliationRelationships.tsv')

PaperId,AuthorId,AffiliationId,AuthorSequenceNumber
122926584,2089499597,157725225,1
122384246,283500697,102335020,2
121824094,2228631194,592451,1
122889433,2167405706,592451,2
122926584,2122007671,157725225,3


#### Get partner affiliations

In [0]:
# Get all partner affiliation Ids
partnerAffiliationIds = partnerPaperAuthorAffiliation \
    .where(partnerPaperAuthorAffiliation.AffiliationId.isNotNull()) \
    .select(partnerPaperAuthorAffiliation.AffiliationId) \
    .distinct()

# Get all partner affiliation details
partnerAffiliations = affiliations \
    .join(partnerAffiliationIds, affiliations.AffiliationId == partnerAffiliationIds.AffiliationId, 'inner') \
    .select(partnerAffiliationIds.AffiliationId, affiliations.DisplayName.alias('AffiliationName'))

display(partnerAffiliations.head(5))

# Save result
asu.save(partnerAffiliations, 'Affiliation/PartnerAffiliations.tsv')

AffiliationId,AffiliationName
57496824,"National Institute of Technology, Arunachal Pradesh"
157417397,Bowling Green State University
45438204,University of Texas at San Antonio
177156846,South Dakota State University
145608581,University of Miami


#### Get partner authors

In [0]:
# Get all partner author Ids
partnerAuthorIds = partnerPaperAuthorAffiliation.select(partnerPaperAuthorAffiliation.AuthorId).distinct()
partnerAuthors = authors \
    .join(partnerAuthorIds, partnerAuthorIds.AuthorId == authors.AuthorId) \
    .select(partnerAuthorIds.AuthorId, authors.DisplayName.alias('AuthorName'))

display(partnerAuthors.head(5))

# Save result
asu.save(partnerAuthors, 'Affiliation/PartnerAuthors.tsv')

AuthorId,AuthorName
1998877,Max Wintermark
16214300,Yassin Refahi
20502297,Iadine Chadès
36625720,David Bodoff
166615085,Nico Görnitz


### Step 7. Display paper counts by year

In [0]:
#Load data from previous output
orgPapers = asu.load('Affiliation/Papers.tsv')

# Get paper count and citation sum for each year
orgPaperGroupByYear = orgPapers \
    .groupBy(orgPapers.Year) \
    .agg(count(orgPapers.PaperId).alias('PaperCount'), sum(orgPapers.CitationCount).alias('CitationSum')) \
    .orderBy(orgPapers.Year)

# Display result
display(orgPaperGroupByYear)

# Save result
asu.save(orgPaperGroupByYear, 'Affiliation/PaperCountByYear.tsv', coalesce=True)

Year,PaperCount,CitationSum
1991,20,3455
1992,70,5187
1993,141,9505
1994,255,21122
1995,351,66893
1996,480,90451
1997,609,83192
1998,813,113911
1999,963,125239
2000,1335,153959
